## INSTALL DEPENDENCIES


### FOR MAC


In [ ]:
%pip install torch

### FOR CUDA (GPU)


In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

### GENERAL


In [ ]:
%pip install presidio_analyzer
%pip install presidio_anonymizer
%pip install transformers
%pip install pandas
%pip install spacy
%pip install spacy-transformers
%pip install tabulate
%pip install multiprocess

### INSTALL SIMPLE SPACY MODEL


In [ ]:
!python -m spacy download en_core_web_sm

### INSTALL COMPLEX SPACY MODEL (ONLY IF YOU USE THIS INSTEAD OF BERT)


In [ ]:
!python -m spacy download en_core_web_lg

## IMPORTS


In [ ]:
import csv
import json
import re
import warnings
from typing import List

import pandas as pd
from presidio_analyzer import RecognizerResult
from spacy import displacy
from tabulate import tabulate
from tqdm import tqdm
from transformers_rec import (
    Analyzer,
    Anonymizer,
    CustomAnalyzerEngine,
)
from utils import fix_entities, model_results, transform_csv_annotated_to_json


## TESTING CUDA FOR GPU


In [ ]:
import torch

torch.cuda.is_available()


## TEST SIMPLE DATA


In [ ]:
def annotate(text: str, analyze_results: List[RecognizerResult]):
    """
    Highlights every identified entity on top of the text.
    :param text: full text
    :param analyze_results: list of analyzer results.
    """
    ents = []

    # Use the anonymizer to resolve overlaps
    anonymizer = Anonymizer(type="simple")
    results = anonymizer.anonymize(text=text, analyze_results=analyze_results)
    # sort by start index
    results = sorted(results.items, key=lambda x: x.start)
    for i, res in enumerate(results):
        ents.append(
            {
                "start": res.start,
                "end": res.end,
                "label": res.entity_type,
                "text": res.text,
            }
        )
    return [{"text": text, "ents": ents}]


In [ ]:
def show_results(an_r, text, analyzer, return_analyzer_results=False ):
    """Show results of analyze() in a dataframe."""
    if (isinstance(analyzer.get_engine(), CustomAnalyzerEngine)):
        df = pd.DataFrame.from_records([r.to_dict() for r in an_r])
        df["text"] = [text[res.start : res.end] for res in an_r]
        df_subset = df[["entity_type", "text", "start", "end", "score"]].rename(
            {
                "entity_type": "Entity type",
                "text": "Text",
                "start": "Start",
                "end": "End",
                "score": "Confidence",
            },
            axis=1,
        )
        df_subset["Text"] = [text[res.start : res.end] for res in an_r]
        #  In analysis_explanation_df there are more columns than in df_subset with more information.
        if return_analyzer_results:
            analysis_explanation_df = pd.DataFrame.from_records(
                [r.analysis_explanation.to_dict() for r in an_r]
            )
        # df_subset = pd.concat([df_subset, analysis_explanation_df], axis=1)
        result = annotate(text, an_r)
        return [(df_subset.reset_index(drop=True), result)]
    else:
        results = []
        for result, text in zip(an_r, text):
            df = pd.DataFrame.from_records([r.to_dict() for r in result])
            df["text"] = [text[res.start : res.end] for res in result]
            df_subset = df[["entity_type", "text", "start", "end", "score"]].rename(
                {
                    "entity_type": "Entity type",
                    "text": "Text",
                    "start": "Start",
                    "end": "End",
                    "score": "Confidence",
                },
                axis=1,
            )
            df_subset["Text"] = [text[res.start : res.end] for res in result]
            #  In analysis_explanation_df there are more columns than in df_subset with more information.
            if return_analyzer_results:
                analysis_explanation_df = pd.DataFrame.from_records(
                    [r.analysis_explanation.to_dict() for r in result]
                )
            # df_subset = pd.concat([df_subset, analysis_explanation_df], axis=1)
            result = annotate(text, result)
            results.append((df_subset.reset_index(drop=True), result))
        return results
            



### SIMPLE TYPE

In [ ]:
analyzer = Analyzer(
    "obi/deid_roberta_i2b2", type="simple"
)  # "en_core_web_lg" or "obi/deid_roberta_i2b2"


In [ ]:
text = "My name is John Smith and I live in New York. My phone number is 212-555-5555 and my email is jhonsmith@gmail.com and my credit card is 4130-2046-7153-3043 and my SSN is 136-84-8867. I am 25 years old and I work at Google."


In [ ]:
analyze_results = analyzer.analyze(
    text=text,
    entities=[
        "PERSON",
        "LOCATION",
        "PHONE_NUMBER",
        "EMAIL_ADDRESS",
        "CREDIT_CARD",
        "US_SSN",
    ],
    language="en",
    score_threshold={
        "PERSON": 0.9,
        "CREDIT_CARD": 0.5,
        "US_SSN": 0.3,
        "EMAIL_ADDRESS": 0.5,
        "PHONE_NUMBER": 0.4,
        "LOCATION": 0.7,
    },
)
print(analyze_results)


### BATCH TYPE

In [ ]:
analyzer = Analyzer("obi/deid_roberta_i2b2", type="batch")


In [ ]:
text = [
    "My name is John Smith and I live in New York. My phone number is 212-555-5555 and my email is jhonsmith@gmail.com and my credit card is 4130-2046-7153-3043 and my SSN is 136-84-8867. I am 25 years old and I work at Google.",
    "As usual, my experience with Mission Belt was just fine. If you need more help, text me at 555-3424-5324",
    "As a customer for 5+ years, we know that a Mission Belt is a great gift, too. Our friend had never seen anything like it. My name is Gonzalo Zelinka",
]


In [ ]:
analyze_results = analyzer.analyze(
    texts=text,
    language="en",
    entities=[
        "PERSON",
        "LOCATION",
        "PHONE_NUMBER",
        "EMAIL_ADDRESS",
        "CREDIT_CARD",
        "US_SSN",
    ],
    score_threshold={
        "PERSON": 0.9,
        "CREDIT_CARD": 0.5,
        "US_SSN": 0.4,
        "EMAIL_ADDRESS": 0.5,
        "PHONE_NUMBER": 0.4,
        "LOCATION": 0.7,
    },
)
print(analyze_results)


### SHOW RESULTS

In [ ]:
if len(analyze_results) > 0:
    results = show_results(analyze_results, text, analyzer=analyzer)
    # print(sentence)
    for result in results: 
        frame, sentence = result
        displacy.render(sentence, style="ent", manual=True)
        display(frame)


## EXTRACT SENTENCES TO SEND THE MODEL


In [ ]:
def extract_sentences_to_eval(input_file, output_file, is_list_reviews=True):
    data = pd.read_csv(input_file, encoding="ISO-8859-1").fillna(method="ffill")
    with open(output_file, "w", encoding="ISO-8859-1") as fo:
        writer = csv.writer(fo)
        num_reviews = 0
        for index, row in data.iterrows():
            if is_list_reviews:
                reviews_list = eval(row["REVIEWS"])
                for review in reviews_list:
                    num_reviews = num_reviews + 1
                    new_row = [
                        str(row["SITE_URL"]),
                        row["ITEM_GROUP_ID"],
                        row["PVIDS"],
                        re.sub(r"[\n\r]{1,}", " ", str(review)),
                    ]
                    writer.writerow(new_row)
            else:
                num_reviews = num_reviews + 1
                new_row = [
                    str(row["SITE_URL"]),
                    row["ITEM_GROUP_ID"],
                    re.sub(r"[\n\r]{1,}", " ", row["PVIDS"]),
                    re.sub(r"[\n\r]{1,}", " ", str(row["REVIEWS"])),
                ]
                writer.writerow(new_row)
        return num_reviews


In [ ]:
num_reviews = extract_sentences_to_eval(
    "testing-data/reviews_from_10_sites.csv",
    "testing-data/cleaned_sentences_from_db3.csv",
    is_list_reviews=False,
)
print(f"Number of reviews: {num_reviews}")


## RUN MODEL


In [ ]:
# because we use n-calls, the warning is annoying as it repeats itself all the time
warnings.filterwarnings(
    "ignore", message="You seem to be using the pipelines sequentially on GPU."
)
model_results(
    input_path="testing-data/cleaned_sentences_from_db2.csv",
    output_path="testing-data/output_from_db3.csv",
)


## EVALUATE MODEL

The format to the dataset if like this:

- [Example Dataset](https://www.kaggle.com/datasets/namanj27/ner-dataset)

Column necessaries and their names:

- Sentence # --> Review #
- Word --> Word
- POS --> Delete this.
- Tag --> Tag.


### EXTRACT THE GROUND TRUTH


In [ ]:
ground_truth = transform_csv_annotated_to_json("testing-data/product_reviews9.csv")
print(ground_truth[1999])


### EXTRACT ONLY SENTENCES TO SEND TO THE MODEL


In [ ]:
def extract_sentences_to_eval(input_file, output_file):
    data = pd.read_csv(input_file, encoding="ISO-8859-1").fillna(method="ffill")
    with open(output_file, "w", encoding="ISO-8859-1") as fo:
        writer = csv.writer(fo)
        writer.writerow(["SENTENCES"])
        for sent, sent_info in data.groupby("Review #"):
            words = list(sent_info["Word"])
            sentence = re.sub(r'\s([?.!"](?:\s|$))', r"\1", " ".join(words))
            writer.writerow([sentence])


In [ ]:
extract_sentences_to_eval(
    "testing-data/product_reviews9.csv", "testing-data/sentences_evaluate.csv"
)


In [ ]:
def extract_sentences_from_db(input_file, output_file):
    data = pd.read_csv(input_file, encoding="ISO-8859-1").fillna(method="ffill")
    reviews = []
    for idx, row in tqdm(data.iterrows(), total=len(data)):
        for review in json.loads(row["REVIEWS"]):
            reviews.append([row["SITE_URL"], row["PVID"], review])
    new_data = pd.DataFrame(reviews, columns=["SITE_URL", "PVID", "REVIEW"])
    new_data.to_csv(output_file, index=False)


In [ ]:
extract_sentences_from_db(
    "testing-data/reviews.csv", "testing-data/sentences_from_db2.csv"
)


### RUN THE MODEL WITH THE EXTRACTED SENTENCES


In [ ]:
model_results(
    analyzer=analyzer, # Always of type batch
    input_path="testing-data/cleaned_sentences_from_db3.csv",
    output_path="testing-data/output_from_db3.csv",
)


### EVALUATING


In [ ]:
with open("testing-data/output_path.json", "r", encoding="ISO-8859-1") as f:
    json_data = f.read()
prediction_data = json.loads(json_data)


In [ ]:
prediction_data[1]


In [ ]:
def calculate_general_scores(entity_scores):
    precision_sum = 0
    recall_sum = 0
    f1_sum = 0
    for entity, scores in entity_scores.items():
        precision_sum += scores["precision"]
        recall_sum += scores["recall"]
        f1_sum += scores["f1"]

    num_entities = len(entity_scores)
    general_precision = precision_sum / num_entities
    general_recall = recall_sum / num_entities
    general_f1 = f1_sum / num_entities

    return general_precision, general_recall, general_f1


#### USE CASE: EXACT

- Identifies the exact words associated with all PII entities in the input text.
- This use case is applicable if the client wants to know which exact words correspond to the PII information. For example to apply masks over the PII entities detected in the input text.


In [ ]:
def calculate_precision_recall_for_entities(
    ground_truth, output_model, unique_entities, output_file
):
    # Initialize counters for true positives, false positives, and false negatives
    tp = {entity: 0 for entity in unique_entities}
    fp = {entity: 0 for entity in unique_entities}
    fn = {entity: 0 for entity in unique_entities}

    for example_idx in range(len(ground_truth)):
        ground_truth_entities = ground_truth[example_idx]["ENTITIES"]
        output_entities = output_model[example_idx]["ENTITIES"]
        # create a set of output entity texts for quick lookup
        ground_truth_texts = set(
            [ent["text"].replace(" ", "") for ent in ground_truth_entities]
        )

        for ground_truth_ent in ground_truth_entities:
            ent_type = ground_truth_ent["entity"]
            if ent_type not in unique_entities:
                continue
            # print(output_entities)
            if any(
                [
                    ent["text"].replace(" ", "")
                    == ground_truth_ent["text"].replace(" ", "")
                    and ent["start"] == ground_truth_ent["start"]
                    for ent in output_entities
                ]
            ):
                tp[ent_type] += 1
            else:
                fn[ent_type] += 1
        for output_ent in output_entities:
            ent_type = output_ent["entity"]
            if ent_type not in unique_entities:
                continue
            if output_ent["text"].replace(" ", "") not in ground_truth_texts:
                fp[ent_type] += 1
    # Calculate precision, recall, and F1 score for each entity
    scores = {}
    table = []
    headers = ["Entity", "Precision", "Recall", "F1"]
    for entity in unique_entities:
        p = (
            round(tp[entity] / (tp[entity] + fp[entity]), 2)
            if tp[entity] + fp[entity] > 0
            else 0
        )
        r = (
            round(tp[entity] / (tp[entity] + fn[entity]), 2)
            if tp[entity] + fn[entity] > 0
            else 0
        )
        f1 = round(2 * p * r / (p + r), 2) if p + r > 0 else 0
        scores[entity] = {"precision": p, "recall": r, "f1": f1}
        table.append([entity, p, r, f1])

    general_precision, general_recall, general_f1 = calculate_general_scores(scores)
    report = f"\nGeneral Precision: {general_precision}\nGeneral Recall: {general_recall}\nGeneral F1: {general_f1}\n{tabulate(table, headers)}"
    with open(output_file, "w", encoding="ISO-8859-1") as f:
        f.write(report)


In [ ]:
calculate_precision_recall_for_entities(
    ground_truth,
    prediction_data,
    set(
        ["PERSON", "EMAIL_ADDRESS", "LOCATION", "US_SSN", "CREDIT_CARD", "PHONE_NUMBER"]
    ),
    "eval/REPORT_LG2.txt",
)


#### USE CASE: BINARY

- Given an input text, it indicates whether each entity (person, location, credit_card, phone_number, us_ssn) is present at least once or not.
- The use case is applicable for filtering out reviews with sensitive information without needing to know which part of the text has the sensitive information.


In [ ]:
def evaluate_entities(ground_truth, output_model, unique_entities, output_file):
    # Initialize counters for true positives, false positives, and false negatives
    tp = {entity: 0 for entity in unique_entities}
    fp = {entity: 0 for entity in unique_entities}
    fn = {entity: 0 for entity in unique_entities}

    # Loop over the documents in the output model
    for i, doc in enumerate(output_model):
        # Get the set of entities present in the document
        model_entity_set = set()
        for entity in doc["ENTITIES"]:
            model_entity_set.add(entity["entity"])
        # Get the set of entities present in the ground truth in the same index of output model
        ground_truth_set = set()
        for entity in ground_truth[i]["ENTITIES"]:
            ground_truth_set.add(entity["entity"])

        # Check if each entity in the ground truth is present in the output model
        for entity in ground_truth_set:
            if entity in model_entity_set:
                # Entity is present in both ground truth and output model
                tp[entity] += 1
            else:
                # Entity is present in ground truth but not in output model
                fn[entity] += 1

        # Check if each entity in the output model is a false positive
        for entity in model_entity_set:
            if entity not in ground_truth_set:
                # Entity is not in the ground truth
                fp[entity] += 1

    # Calculate precision, recall, and F1 score for each entity
    scores = {}
    table = []
    headers = ["Entity", "Precision", "Recall", "F1"]
    for entity in unique_entities:
        p = (
            round(tp[entity] / (tp[entity] + fp[entity]), 2)
            if tp[entity] + fp[entity] > 0
            else 0
        )
        r = (
            round(tp[entity] / (tp[entity] + fn[entity]), 2)
            if tp[entity] + fn[entity] > 0
            else 0
        )
        f1 = round(2 * p * r / (p + r), 2) if p + r > 0 else 0
        scores[entity] = {"precision": p, "recall": r, "f1": f1}
        table.append([entity, p, r, f1])

    general_precision, general_recall, general_f1 = calculate_general_scores(scores)
    report = f"\nGeneral Precision: {general_precision}\nGeneral Recall: {general_recall}\nGeneral F1: {general_f1}\n{tabulate(table, headers)}"
    with open(output_file, "w", encoding="ISO-8859-1") as f:
        f.write(report)


In [ ]:
# a1 = [{'TEXT': 'a1', "ENTITIES": [{'start': 21, 'end': 33, 'entity': 'PERSON', 'text': 'Zoey Edwards'}, {'start': 129, 'end': 151, 'entity': 'EMAIL_ADDRESS', 'text': 'edwards-zoey@gmail.com'}, {'start': 190, 'end': 201, 'entity': 'LOCATION', 'text': '900 F St NW'}, {'start': 213, 'end': 222, 'entity': 'US_SSN', 'text': '367245504'}, {'start': 252, 'end': 271, 'entity': 'CREDIT_CARD', 'text': '2259-8740-7030-1462'}]}]
# a2 = [{"TEXT": "a2", "ENTITIES": [{'start': 21, 'end': 33, 'entity': 'PERSON', 'text': 'Zoey Edwards'}, {'start': 129, 'end': 151, 'entity': 'EMAIL_ADDRESS', 'text': 'edwards-zoey@gmail.com'}, {'start': 190, 'end': 195, 'entity': 'LOCATION', 'text': '900 F'}, {'start': 196, 'end': 201, 'entity': 'LOCATION', 'text': 'St NW'}, {'start': 213, 'end': 222, 'entity': 'US_SSN', 'text': '367245504'}, {'start': 252, 'end': 255, 'entity': 'PHONE_NUMBER', 'text': '225'}, {'start': 255, 'end': 271, 'entity': 'PHONE_NUMBER', 'text': '9-8740-7030-1462'}]}]
evaluate_entities(
    ground_truth,
    prediction_data,
    set(
        ["PERSON", "EMAIL_ADDRESS", "LOCATION", "US_SSN", "CREDIT_CARD", "PHONE_NUMBER"]
    ),
    "eval/REPORT_SENTENCE_LG.txt",
)
